In [19]:
import json
import nltk.data
import re

In [13]:
with open('C:\\Users\\achang\\Documents\\jupyter\\data\\simplewiki\\simplewiki-20171103.parsed.json') as f:
    wiki = json.load(f)

In [18]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [39]:
def print_stats(wiki):
    num_with = 0
    num_without = 0
    num_links = 0
    for title, page in wiki.items():
        sentences = sentence_tokenizer.tokenize(page['text'])
        for sentence in sentences:
            if re.search(r'\{\{\d+\}\}', sentence):
                num_with += 1
            else:
                num_without += 1
        num_links += len(page['links'])
    print('with:', num_with)
    print('without:', num_without)
    print('links:', num_links)

In [40]:
print_stats(wiki)

with: 539837
without: 555318
links: 1055666


In [91]:
def parse_sentences(wki):
    result = []
    for title, page in wiki.items():
        sentences = sentence_tokenizer.tokenize(page['text'])
        for sentence in sentences:
            src_offset = 0
            dst_offset = 0
            links = []
            strs = []
            
            for match in re.finditer(r'\{\{\d+\}\}', sentence):
                token = sentence[match.start():match.end()]
                link = page['links'][token]
                
                src_fragment = sentence[src_offset:match.start()]
                
                link_target = link['target']
                link_text = link['text']
                link_start = dst_offset + len(src_fragment)
                link_finish = link_start + len(link_text)
                
                strs.append(src_fragment)
                strs.append(link_text)
                
                links.append({
                    'start': link_start,
                    'finish': link_finish,
                    'target': link_target
                })
                
                src_offset = match.end()
                dst_offset += len(src_fragment) + len(link_text)
                
            strs.append(sentence[src_offset:])
                
            result.append({
                'text': ''.join(strs),
                'links': links
            })
    return result

In [92]:
sentences = parse_sentences(wiki)

In [103]:
with open('C:\\Users\\achang\\Documents\\jupyter\\data\\simplewiki\\simplewiki-20171103.sentences.json', 'w') as f:
    json.dump(sentences, f, indent = 1)